## Aula 05 - Filtragem Híbrida - Exercícios

In [1]:
import pandas as pd
import numpy as np

### Importar base de dados

In [2]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar.gz
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/ratings_sample.csv


In [6]:
# ==== SETUP (autocontido) ====
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

# 1) Baixar e carregar dados (idempotente)
import os, subprocess, sys
if not os.path.exists("dataset/movies_sample.csv"):
    try:
        import wget  # noqa
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "wget", "-q"])
    subprocess.check_call([sys.executable, "-m", "wget", "https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz"])
    subprocess.check_call(["tar", "-xvzf", "ml-20m-compact.tar.gz"])

movies = pd.read_csv("./dataset/movies_sample.csv")
ratings = pd.read_csv("./dataset/ratings_sample.csv")

# 2) df com título (IDs originais)
df_raw = ratings[['userId', 'movieId', 'rating']].merge(movies[['movieId','title']])

# 3) mapear para IDs internos (necessário p/ matrizes)
map_users = {u:i for i,u in enumerate(df_raw.userId.unique())}
map_items = {m:i for i,m in enumerate(df_raw.movieId.unique())}

df = df_raw.copy()
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)

# 4) train/test
train, test = train_test_split(df, test_size=0.2, random_state=2)

# 5) movies_genres com IDs internos
movies_genres = movies.drop('genres', axis=1).join(
    movies.genres.str.split('|', expand=True).stack().reset_index(drop=True, level=1).rename('genre')
)
movies_genres['movieId'] = movies_genres['movieId'].map(map_items)
movies_genres.dropna(inplace=True)
movies_genres['movieId'] = movies_genres['movieId'].astype(int)

# 6) estruturas auxiliares para os exercícios
n_items = int(df.movieId.max()) + 1

# ratings por item e por usuário (usando apenas TREINO)
item_ratings = defaultdict(dict)
user_ratings = defaultdict(dict)
for u, i, r in train[['userId','movieId','rating']].itertuples(index=False):
    item_ratings[int(i)][int(u)] = float(r)
    user_ratings[int(u)][int(i)] = float(r)

# gêneros por item
genres_by_item = defaultdict(list)
for _, row in movies_genres[['movieId','genre']].iterrows():
    genres_by_item[int(row.movieId)].append(str(row.genre))

# util: títulos por item interno
title_by_id = {int(row.movieId): row.title for _, row in df[['movieId','title']].drop_duplicates().iterrows()}

print("SETUP OK ✅")
print(f"- Usuários: {len(map_users)} | Itens: {len(map_items)} | n_items (interno): {n_items}")
print(f"- Train: {len(train)} | Test: {len(test)}")


SETUP OK ✅
- Usuários: 11090 | Itens: 417 | n_items (interno): 417
- Train: 152496 | Test: 38125


### Obs. Nesta aula, você poderá escolher entre implementar o exercício 1 **OU** o exercício 2.

***Exercício 01:*** Implemente uma hibridização monolítica/combinação usando a seguinte heurística:
- Uso do algoritmo ItemAtributeKNN, sendo a hibridização feita no cálculo das similaridades entre os itens.
- Se a quantidade de usuários que avaliaram ambos os itens for maior que um limiar L1, calcule a similaridade entre esses itens usando cosseno aplicado à representação baseada em notas.
- Caso contrário, calcule a similaridade entre os itens usando metadados (gêneros por exemplo).  

Compare os resultados do algoritmo híbrido com as versões isoladas do mesmo algoritmo.

In [7]:
import numpy as np
from collections import defaultdict

# ---------- Utilidades ----------
def mae_rmse(y_true, y_pred):
    y_true = np.asarray(y_true, float); y_pred = np.asarray(y_pred, float)
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = float(np.sqrt(np.mean((y_true - y_pred) ** 2)))
    return mae, rmse

def cosine_on_overlap(vec_i, vec_j):
    # vec_* são dict {user: rating}. Cálculo no espaço dos usuários em comum.
    users = set(vec_i) & set(vec_j)
    if not users:
        return 0.0
    xi = np.array([vec_i[u] for u in users], dtype=float)
    xj = np.array([vec_j[u] for u in users], dtype=float)
    num = float(np.dot(xi, xj))
    den = float(np.linalg.norm(xi) * np.linalg.norm(xj))
    return (num / den) if den != 0 else 0.0

def jaccard(a, b):
    A, B = set(a), set(b)
    return len(A & B) / len(A | B) if (A or B) else 0.0

# ---------- Estruturas: ratings por item/usuário e gêneros por item ----------
n_items = int(df.movieId.max()) + 1

# ratings por item: {item: {user: rating}}
item_ratings = defaultdict(dict)
# ratings por user: {user: {item: rating}} (útil depois)
user_ratings = defaultdict(dict)
for u, i, r in train[['userId','movieId','rating']].itertuples(index=False):
    item_ratings[int(i)][int(u)] = float(r)
    user_ratings[int(u)][int(i)] = float(r)

# gêneros por item (se necessário, derive como na Aula 04)
if 'movies_genres' not in globals():
    movies_genres = movies.drop('genres', axis=1).join(
        movies.genres.str.split('|', expand=True).stack().reset_index(drop=True, level=1).rename('genre')
    )
    movies_genres['movieId'] = movies_genres['movieId'].map({m:i for i,m in enumerate(df.merge(movies[['movieId']])['movieId'].unique())})
    movies_genres.dropna(inplace=True)
    movies_genres['movieId'] = movies_genres['movieId'].astype(int)

genres_by_item = defaultdict(list)
for _, row in movies_genres[['movieId','genre']].iterrows():
    genres_by_item[int(row.movieId)].append(str(row.genre))

# ---------- Matrizes de similaridade: CF (cosseno), Conteúdo (Jaccard), Híbrida ----------
def build_S_cf(n_items, item_ratings, min_overlap=1):
    S = np.zeros((n_items, n_items), dtype=np.float32)
    for i in range(n_items):
        vi = item_ratings.get(i, {})
        for j in range(i+1, n_items):
            vj = item_ratings.get(j, {})
            overlap = len(set(vi) & set(vj))
            if overlap >= min_overlap:
                s = cosine_on_overlap(vi, vj)
                if s > 0:
                    S[i, j] = S[j, i] = s
    np.fill_diagonal(S, 1.0)
    return S

def build_S_content(n_items, genres_by_item):
    S = np.zeros((n_items, n_items), dtype=np.float32)
    for i in range(n_items):
        gi = genres_by_item.get(i, [])
        for j in range(i+1, n_items):
            gj = genres_by_item.get(j, [])
            s = jaccard(gi, gj)
            if s > 0:
                S[i, j] = S[j, i] = s
    np.fill_diagonal(S, 1.0)
    return S

def build_S_hybrid(n_items, item_ratings, genres_by_item, L1=5):
    # Se overlap >= L1 -> cosseno (CF), senão -> Jaccard (conteúdo)
    S = np.zeros((n_items, n_items), dtype=np.float32)
    for i in range(n_items):
        vi = item_ratings.get(i, {})
        gi = genres_by_item.get(i, [])
        for j in range(i+1, n_items):
            vj = item_ratings.get(j, {})
            gj = genres_by_item.get(j, [])
            overlap = len(set(vi) & set(vj))
            if overlap >= L1:
                s = cosine_on_overlap(vi, vj)
            else:
                s = jaccard(gi, gj)
            if s > 0:
                S[i, j] = S[j, i] = s
    np.fill_diagonal(S, 1.0)
    return S

print("Construindo S_cf (min_overlap=1) ...")
S_cf = build_S_cf(n_items, item_ratings, min_overlap=1)

print("Construindo S_content (Jaccard gêneros) ...")
S_content = build_S_content(n_items, genres_by_item)

L1 = 5  # limiar de usuários em comum
print(f"Construindo S_hybrid (L1={L1}) ...")
S_hybrid = build_S_hybrid(n_items, item_ratings, genres_by_item, L1=L1)

# ---------- Predição item-based KNN ----------
def knn_predict(uid, iid, S, k=10):
    uid, iid = int(uid), int(iid)
    hist = user_ratings.get(uid, {})
    if not hist or iid >= S.shape[0]:
        return 3.0
    sims = []
    for j, r in hist.items():
        if j < S.shape[1]:
            s = float(S[iid, j])
            if s > 0:
                sims.append((s, r))
    sims.sort(key=lambda x: x[0], reverse=True)
    sims = sims[:k]
    if not sims:
        # fallback: média do usuário
        return float(np.mean(list(hist.values())))
    num = sum(s*r for s, r in sims)
    den = sum(s for s, _ in sims)
    return (num / den) if den != 0 else float(np.mean(list(hist.values())))

def evaluate_on(S, k=10):
    preds = [knn_predict(u, i, S, k=k) for u, i in zip(test.userId, test.movieId)]
    return mae_rmse(test.rating, preds)

# ---------- Avaliar e comparar ----------
for k in [5, 10, 20]:
    mae_cf, rmse_cf = evaluate_on(S_cf, k=k)
    mae_ct, rmse_ct = evaluate_on(S_content, k=k)
    mae_hy, rmse_hy = evaluate_on(S_hybrid, k=k)
    print(f"\nK={k}")
    print(f"  CF (cosseno)     -> MAE={mae_cf:.4f}  RMSE={rmse_cf:.4f}")
    print(f"  Conteúdo (gênero)-> MAE={mae_ct:.4f}  RMSE={rmse_ct:.4f}")
    print(f"  Híbrido (L1={L1}) -> MAE={mae_hy:.4f}  RMSE={rmse_hy:.4f}")

print("\nInterpretação rápida:")
print("- Com L1 maior, a matriz híbrida privilegia CF quando há bastante sobreposição;")
print("  para pares raros, recorre a conteúdo, cobrindo sparsidade. Em geral melhora RMSE.")


Construindo S_cf (min_overlap=1) ...
Construindo S_content (Jaccard gêneros) ...
Construindo S_hybrid (L1=5) ...

K=5
  CF (cosseno)     -> MAE=0.7427  RMSE=0.9779
  Conteúdo (gênero)-> MAE=0.8236  RMSE=1.0676
  Híbrido (L1=5) -> MAE=0.7409  RMSE=0.9756

K=10
  CF (cosseno)     -> MAE=0.7477  RMSE=0.9743
  Conteúdo (gênero)-> MAE=0.8089  RMSE=1.0484
  Híbrido (L1=5) -> MAE=0.7470  RMSE=0.9736

K=20
  CF (cosseno)     -> MAE=0.7645  RMSE=0.9861
  Conteúdo (gênero)-> MAE=0.8064  RMSE=1.0453
  Híbrido (L1=5) -> MAE=0.7642  RMSE=0.9859

Interpretação rápida:
- Com L1 maior, a matriz híbrida privilegia CF quando há bastante sobreposição;
  para pares raros, recorre a conteúdo, cobrindo sparsidade. Em geral melhora RMSE.


***Exercício 02:*** Vamos implementar um recomendador híbrido canalizado em cascata, no cenário de ranqueamento. A ideia é que um primeiro algoritmo gere uma lista C1 de N=50 itens candidatos à recomendação para cada usuário. Em seguida, um outro recomendador irá gerar uma outra lista C2 também de N=50 itens candidatos à rcomendação para cada usuário. Por fim, o ranking final será a intersecção entre C1 e C2, sendo o score de cada itens formado pela média aritmética dos scores de cada lista. Avalie o desempenho.

Dica 1: utilize o parâmetro rank_length disponível nos algoritmos de ranqueamento do CaseRecommender para especificar o tamanho N de recomendações para cada usuário.

Dica 2: você pode gravar num arquivo os rankings gerados por um algoritmo para cada usuário especificando o nome do arquivo no parâmetro output_file.

Dica 3: consulte a Aula 04 que contém algumas métricas de avaliação de ranqueamento. Como você irá gerar o ranking final externamente ao CaseRecommender, será necessário avaliá-lo usando funções próprias.

In [8]:
import numpy as np
from collections import defaultdict

# Reusa S_cf e S_content do Ex. 01 (se não existirem, crie-os antes)

# ---------- Scoring por usuário ----------
def score_items_for_user(uid, S, candidate_items):
    uid = int(uid)
    hist = user_ratings.get(uid, {})
    if not hist:
        return {i: 0.0 for i in candidate_items}
    scores = {}
    for iid in candidate_items:
        s_list = []
        for j, r in hist.items():
            if iid < S.shape[0] and j < S.shape[1]:
                s = float(S[iid, j])
                if s > 0:
                    s_list.append((s, r))
        if s_list:
            num = sum(s*r for s, r in s_list)
            den = sum(s for s, _ in s_list)
            scores[iid] = (num/den) if den != 0 else np.mean(list(hist.values()))
        else:
            scores[iid] = np.mean(list(hist.values()))
    return scores

def unseen_items(uid, n_items):
    seen = set(user_ratings.get(int(uid), {}).keys())
    return [i for i in range(n_items) if i not in seen]

def topN_from_scores(scores_dict, N):
    # retorna lista ordenada de (item, score)
    items_scores = sorted(scores_dict.items(), key=lambda x: x[1], reverse=True)
    return items_scores[:N]

# ---------- Métricas de ranking ----------
def eval_ranking(users, rankings, test_df, K=10, rel_threshold=3.0):
    # rankings: dict {u: [item1,item2,...]}
    # ground truth: relevantes no test (rating >= threshold)
    test_rel = defaultdict(set)
    for u, i, r in test_df[['userId','movieId','rating']].itertuples(index=False):
        if r >= rel_threshold:
            test_rel[int(u)].add(int(i))

    precisions, recalls, ndcgs, aps = [], [], [], []

    def dcg(rels):
        return sum((rel / np.log2(idx+2)) for idx, rel in enumerate(rels))

    for u in users:
        rec_list = rankings.get(u, [])[:K]
        if not rec_list:
            continue
        relset = test_rel.get(u, set())
        hits = [1.0 if i in relset else 0.0 for i in rec_list]

        # Precision@K / Recall@K
        P = sum(hits) / len(rec_list)
        R = (sum(hits) / len(relset)) if relset else 0.0

        # nDCG@K
        dcg_val = dcg(hits)
        ideal_hits = sorted(hits, reverse=True)
        idcg_val = dcg(ideal_hits) if ideal_hits else 1.0
        nDCG = dcg_val / idcg_val if idcg_val > 0 else 0.0

        # AP@K
        cum, ap_sum = 0.0, 0.0
        for idx, h in enumerate(hits, 1):
            if h > 0:
                cum += 1
                ap_sum += cum / idx
        AP = (ap_sum / sum(hits)) if sum(hits) > 0 else 0.0

        precisions.append(P); recalls.append(R); ndcgs.append(nDCG); aps.append(AP)

    def avg(x): return float(np.mean(x)) if x else 0.0
    return {
        "P@K": avg(precisions),
        "R@K": avg(recalls),
        "nDCG@K": avg(ndcgs),
        "MAP@K": avg(aps),
        "Users@K": len(precisions)
    }

# ---------- Pipeline em Cascata ----------
N = 50      # tamanho da lista de candidatos de cada estágio
K = 10      # corte para avaliação final
users = sorted(user_ratings.keys())
n_items_total = int(df.movieId.max()) + 1

rank_stage1 = {}   # CF
rank_stage2 = {}   # Conteúdo
rank_cascade = {}  # Interseção + média

for u in users:
    cands = unseen_items(u, n_items_total)

    # Stage 1 (CF)
    scores1 = score_items_for_user(u, S_cf, cands)
    top1 = topN_from_scores(scores1, N)
    rank_stage1[u] = [i for i, _ in top1]

    # Stage 2 (Conteúdo)
    scores2 = score_items_for_user(u, S_content, cands)
    top2 = topN_from_scores(scores2, N)
    rank_stage2[u] = [i for i, _ in top2]

    # Cascata: interseção + média (normalizada por usuário para equilíbrio)
    set1, set2 = set(rank_stage1[u]), set(rank_stage2[u])
    inter = list(set1 & set2)

    if inter:
        # normalização min-max por usuário em cada estágio
        def normalize(scores, items):
            vals = np.array([scores[i] for i in items], float)
            vmin, vmax = vals.min(), vals.max()
            return {i: (scores[i]-vmin)/(vmax-vmin+1e-12) for i in items}

        norm1 = normalize(scores1, inter)
        norm2 = normalize(scores2, inter)
        fused = {i: 0.5*norm1[i] + 0.5*norm2[i] for i in inter}
        rank_cascade[u] = [i for i, _ in sorted(fused.items(), key=lambda x: x[1], reverse=True)[:K]]
    else:
        # fallback: usa Stage 1
        rank_cascade[u] = rank_stage1[u][:K]

# ---------- Avaliação (comparar 3 listas) ----------
res1 = eval_ranking(users, rank_stage1, test, K=K)
res2 = eval_ranking(users, rank_stage2, test, K=K)
resc = eval_ranking(users, rank_cascade, test, K=K)

print(f"\nAvaliação @K={K} (usuários avaliados: {resc['Users@K']})")
print("Stage 1 (CF):        ", {k: round(v,4) for k,v in res1.items() if k!='Users@K'})
print("Stage 2 (Conteúdo):  ", {k: round(v,4) for k,v in res2.items() if k!='Users@K'})
print("Cascata (Interseção):", {k: round(v,4) for k,v in resc.items() if k!='Users@K'})

print("\nNotas:")
print("- A cascata tende a aumentar a precisão/diversidade ao exigir concordância entre os modelos.")
print("- Se a interseção for pequena, o fallback usa o Stage 1. Você pode trocar por união com penalidade.")



Avaliação @K=10 (usuários avaliados: 11090)
Stage 1 (CF):         {'P@K': 0.0001, 'R@K': 0.0001, 'nDCG@K': 0.0002, 'MAP@K': 0.0002}
Stage 2 (Conteúdo):   {'P@K': 0.0106, 'R@K': 0.0364, 'nDCG@K': 0.0551, 'MAP@K': 0.0416}
Cascata (Interseção): {'P@K': 0.0001, 'R@K': 0.0001, 'nDCG@K': 0.0002, 'MAP@K': 0.0001}

Notas:
- A cascata tende a aumentar a precisão/diversidade ao exigir concordância entre os modelos.
- Se a interseção for pequena, o fallback usa o Stage 1. Você pode trocar por união com penalidade.
